In [ ]:
import pandas as pd
df1 = pd.read_csv("/opt/ml/code/objectDetection_iseo/yolov5/Kfold/exp11/output_s.csv")
df2 = pd.read_csv("/opt/ml/code/objectDetection_iseo/yolov5/Kfold/exp12/submission1.csv")
df3 = pd.read_csv("/opt/ml/code/objectDetection_iseo/yolov5/Kfold/exp11/output_s.csv")

In [ ]:
# Yolo filename -> basic file name
for i, fn in enumerate(df2.image_id.values):
    df3.loc[(df3["image_id"]==fn),"PredictionString2"]=df2.loc[i,"PredictionString"]
    


In [ ]:
def pre_processing(predictions, image_size=512):
    boxes = []
    scores = []
    labels = []
    for i, pred in enumerate(predictions):
        if i%6 ==0:
            box = []         
            if pred == '':
                break
            labels.append(int(pred))
        elif i%6 ==1:
            scores.append(float(pred))
                    
        elif i%6==2 or i%6==3 or i%6==4 :
            box.append(float(pred)/image_size)
        else:
            box.append(float(pred)/image_size)
            boxes.append(box)
            
    return boxes, scores, labels

In [ ]:
from ensemble_boxes import *
def ensemble_wbf(df1,df2, weights=None, iou_thr=0.6, skip_box_thr=0.75):
    for si in range(len(df1)):
        boxes1, scores1, labels1 = pre_pc(df1.loc[si,"PredictionString"].split(' '))
        boxes2, scores2, labels2 = pre_pc(df2.loc[si,"PredictionString"].split(' '))

        boxes_list=[boxes1, boxes2 for ]
        scores_list=[]
        labels_list=[]
        boxes_list.append(boxes1)
        boxes_list.append(boxes2)
        scores_list.append(scores1)
        scores_list.append(scores2)
        labels_list.append(labels1)
        labels_list.append(labels2)

        boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list, weights=None, iou_thr=0.5, skip_box_thr=0.2)
        boxes=boxes*512
        pre_str=''
        for i in range(len(labels)):
            pre_str+=str(int(labels[i]))+' '+str(scores[i])+' '+str(boxes[i][0])+' '+str(boxes[i][1])+' '+str(boxes[i][2])+' '+str(boxes[i][3])+' ' 
        df1.loc[si, "PredictionString"]=pre_str


